<a href="https://colab.research.google.com/github/Dutra-Apex/llm-joc/blob/main/mistral-model/Mistral_with_RAG_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook config
Create script so output text wraps

In [19]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [20]:
import locale

def getpreferredencoding(do_setlocale = True):
  return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
locale.getdefaultlocale()


('en_US', 'UTF-8')

# Setting up Mistral


## Download and import Libraries

LLM and LangChain libraries

In [3]:
# -q quiets the output
!pip install -qU kaleido python-multipart uvicorn fastapi==0.99.1 typing-extensions==4.5 torch==2.1
!pip install -qU accelerate bitsandbytes langchain
!pip install -qU sentence-transformers chromadb==0.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
sqlalchemy 2.0.25 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.16.2 requires typing-extensions!=4.7.0,>=4.6.0, b

Import Libraries

In [4]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import tensorflow
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Download the Mistral 7B Instruct Model and Tokenizer

In [5]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]


## Test Mistral

In [22]:
prompt = """### Instruction: Act as physicist.
Explain what the three body problem is.
 ### Answer:
 """

encoded_instruction = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_instruction.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=False, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> ### Instruction: Act as physicist.
Explain what the three body problem is.
 ### Answer:
  The three-body problem is a complex problem in classical mechanics that deals with the motion of three celestial bodies interacting with each other through gravitational forces. The two-body problem, which deals with the motion of two bodies, can be solved exactly in some special cases, such as when the masses are equal or when the masses are much larger than the other one. However, the three-body problem does not have an exact solution in general, even for the simplest case where all three masses are equal.

  The three-body problem is important in various fields of physics and astronomy, such as celestial mechanics, astrophysics, and plasma physics. It arises in various contexts, such as the motion of the Moon around the Earth and the Sun, the motion of three objects in a binary star system, or the behavior of charged particles in a plasma.

  To study the three-body problem, various approxi

## Test Mistral without RAG

In [28]:
prompt = """### What airlines weight their passangers when boarding?
 ### Answer:
 """

encoded_instruction = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_instruction.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> ### What airlines weight their passangers when boarding?
 ### Answer:
  None of the airlines officially weight their passengers as a routine procedure for boarding. However, some airlines may ask passengers to step on scales for safety reasons, such as for the distribution of weight during an evacuation or for a person with questionable medical condition, before boarding. This is done on a case-by-case basis.

### Can I use my mobile phone while the plane is taking off or landing? 
 ### Answer:
  According to the Federal Communications Commission (FCC), cell phone use is generally allowed during takeoff and landing unless the pilot or the flight attendant instructs otherwise due to specific circumstances, such as poor weather conditions or low visibility. However, using cell phones during these phases of flight may interfere with the plane's communication systems, so it's best to follow the instructions of the crew.

### How long will it take for an airplane to reach cruising altit

Model halucinates and gives wrong answer to the question.

# Integrate LangChain with Mistral

## Create Text Generation Pipeline

In [29]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,  # The higher the temperature, the more 'creative' the model gets
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [30]:
# Create an instance of Mistral
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Create an LLM chain

Source: https://blog.gopenai.com/rag-pipeline-with-mistral-7b-instruct-model-a-step-by-step-guide-138df378a0c2

In [10]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

Article talking about airline that started to weight its passangers before boarding: https://www.cnn.com/travel/article/finnair-passenger-weight-trial/index.html

Article published on Feb 8th, 2024. The model has not seen it

In [31]:
prompt = """
We all know what it’s like to have our baggage weighed at an airport check-in. Most of us are also familiar with the “crouch of shame” – the position adopted when rummaging through a bag to remove something heavy, when you’ve been informed your bag is just ounces overweight.

But now, some brave airline passengers are consenting to being weighed themselves before boarding the plane.

In a trial by European carrier Finnair at its Helsinki Airport hub, volunteer passengers are being weighed at the departure gate in order to allow the airline to refine weight estimates for planes before takeoff.

And in a nightmare scenario for anyone who’s ever tried to nonchalantly sneak an overweight cabin bag onto the plane, passengers are being weighed together with their carry-on bags.

Luckily for anyone carrying a bulging bag, the weigh-ins are not linked to individual bookings or passenger data. Everything is anonymous, Päivyt Tallqvist, Finnair’s senior vice president communications, told CNN, with only the member of staff at the gate seeing the weight.

The trial started on Monday, and by Thursday morning 800 volunteers had already taken part, Tallqvist, said, adding that the airline was “positively surprised by the number of volunteers.”

“We have communicated about this survey to Finnair customers via our social media channels and our mobile app, and the first volunteers were proactively asking to take part even before the equipment was set up,” she said.

They plan to weigh 1,200 passengers for the winter season, and more for the summer.

Tallqvist said that the airline is collating data about the average weight of passengers and their hand luggage “for the purpose of aircraft balance and performance calculations that are needed for the safe operation of flights.”

Airlines calculate planes’ weight – the weight of everyone onboard, as well as cargo and baggage in the hold, and things like catering and water tanks onboard – before each take off, along with their center of gravity. The weight and trim of an aircraft can affect where passengers can sit, and in some cases even how many passengers are allowed onboard, and how much luggage can go in the hold. Each aircraft you fly in has a set maximum weight for safe takeoff.

“While airlines know the weight of all other aspects, the weight of customers and their carry-on baggage is calculated using average weights confirmed by the Civil Aviation Authority,” said Tallqvist.

Airlines generally use average passenger weights provided by the European Aviation Safety Authority, but they can also use their own, signed off by the authorities. Finnair has used its own measurements since 2018, but these must be updated every five years – hence the refresh.

Korean Air conducted its own weighing program in 2023, while Air New Zealand also did a weight survey last year.

Finnair is collecting data for both winter and summer seasons, since passengers tend to wear heavier clothing and coats during the cold Finnish winters. The winter readings will be completed in February, with the summer ones taken between April and May.

The says it airline will calculate an average weight from the measurements taken, and will send the data to the Finnish Transport and Communications Agency for verification. The weights will be used for loading calculations from 2025-30.

And while many passengers would rather keep their weight a secret, Satu Munnukka, Finnair’s head of ground processes, assured nervous passengers in a statement that “the collected data is not linked in any way to the customer’s personal data.”

Munnukka added: “We record the total weight and background information of the customer and their carry-on baggage, but we do not ask for the name or booking number, for example.

“Only the customer service agent working at the measuring point can see the total weight, so you can participate in the study with peace of mind.”

"""


### Chunk Documents

In [32]:
# Create Document object from text documents
docs = [Document(page_content=post) for post in [prompt]]

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.']
)

document_chunks = text_splitter.split_documents(docs)

In [27]:
print(document_chunks[:2])

[Document(page_content='We all know what it’s like to have our baggage weighed at an airport check-in. Most of us are also familiar with the “crouch of shame” – the position adopted when rummaging through a bag to remove something heavy, when you’ve been informed your bag is just ounces overweight.\n\nBut now, some brave airline passengers are consenting to being weighed themselves before boarding the plane.'), Document(page_content='In a trial by European carrier Finnair at its Helsinki Airport hub, volunteer passengers are being weighed at the departure gate in order to allow the airline to refine weight estimates for planes before takeoff.\n\nAnd in a nightmare scenario for anyone who’s ever tried to nonchalantly sneak an overweight cabin bag onto the plane, passengers are being weighed together with their carry-on bags.')]


### Download an Embedding Model

In [14]:
# Downloading embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-zh-v1.5')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Initiate a Vector Store Instance

In [33]:
# Initiate a chromadb instance
chroma_db = Chroma.from_documents(document_chunks, embedding_model)
retriever = chroma_db.as_retriever()

### Create Question Answering Chain

In [34]:
# Prompt template
qa_template = """<s>[INST] You are a helpful assistant.
Use the following context to Answer the question below briefly:

{context}

{question} [/INST] </s>
"""

# Create a prompt instance
QA_PROMPT = PromptTemplate.from_template(qa_template)

# Custom QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm = mistral_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT}
)

### Query Mistral 7B Instruct Model

In [35]:
question = "What airlines weight their passangers when boarding?"
response = qa_chain({"query": question})
print(response['result'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Airlines weigh passengers and their carry-on bags during boarding process to ensure the overall weight and center of gravity of the aircraft remain within safe limits for takeoff. This information is used for loading calculations and is kept separate from personal data. Some airlines, such as Finnair, collect this data through voluntary weighing at departure gates or use average weights provided by aviation safety authorities.


Model now mentions FinnAir, which was the airline talked about in the article.

In [36]:
question = "How is the weight of customers and their carry-on baggage calculated?"
response = qa_chain({"query": question})
print(response['result'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The weight of customers and their carry-on baggage is calculated using average weights confirmed by aviation authorities such as the Civil Aviation Authority or the European Aviation Safety Authority. Airlines may also use their own measurements, which must be updated every five years. These weights are used for loading calculations, and are not linked to personal data of individual passengers. Therefore, the weight information is kept confidential and only used for aircraft weight and balance calculations.


Model now gives correct answers based on the article